### Simulation version 1

Errors:
* Does not find parent neighbours if not connected to parent

In [ ]:

class JacksonSimulation():

    def __init__(self, initial_n, T, m, pm, n, pn, p_SES_high):

        # parameters
        self.initial_n = initial_n
        self.T = T
        self.m = m
        self.pm = pm
        self.n = n
        self.pn = pn
        self.p_SES_high = p_SES_high # random for now, could be determined by the parent nodes

        # also maybe add options for the functions used
        # self.initialiser_function = 

        # simulation memory
        self.period = 0
        self.graph_history = []
        # will also need a matrix history to make it easier to save simulations to a csv or the like

        # starting stuff
        #self.initial_nodes = [i+1 for i in range(initial_n)] # create n initial nodes, +1 for name to start at 1

        self.main()


    '''MAIN'''
    def main(self):
        # make the initial graph
        self.save_update_to_memory(self.initial_node_connector_basic())


        for t in range(self.T):
            self.update_simulation(self.graph_history[-1])



    '''INITIALISATION'''
    # basic version: all nodes get a certain number of connections
    def initial_node_connector_basic(self):
        initial_graph = nx.Graph()

        # add nodes until we have enough to start with
        while len(initial_graph.nodes) < self.initial_n:
            new_node = self.new_node_birth(initial_graph)
            initial_graph.add_nodes_from([new_node])


        #initial_graph.add_nodes_from(self.initial_nodes)

        initial_nodes = list(initial_graph.nodes)


        for node in initial_nodes:
            possible_links = initial_nodes.copy()
            possible_links.remove(node) # remove own node
            
            # randomly chooses m+n nodes without replacement from the initial nodes to connect (correction for taking the number of nodes instead if m+n>initial_n)
            initial_neighbours = list(np.random.choice(possible_links, min(self.m + self.n, self.initial_n-1), replace=False))

            # creates the edges
            initial_edges = self.helper_functions().edge_creator_helper(node, initial_neighbours)
            initial_graph.add_edges_from(initial_edges)

        return initial_graph


    class basic_world:





        pass

    '''UPDATING'''
    # big simulation update
    def update_simulation(self, current_graph):

        # add the new node
        new_node = self.new_node_birth(current_graph) # DONE

        # get the connections for the new node
        parent_connections = self.find_parent_nodes_connections(current_graph) # DONE
        parent_neighbour_connections = self.find_parent_neighbour_connections(current_graph, parent_connections) # DONE
             
        # combine the two
        new_node_connections = np.concatenate((parent_connections, parent_neighbour_connections))
        new_node_edges = self.helper_functions().edge_creator_helper(new_node[0], new_node_connections)

        # copy graph with the added connections
        new_graph = current_graph.copy()

        new_graph.add_nodes_from([new_node]) #issue is here
        new_graph.add_edges_from(new_node_edges)

        self.save_update_to_memory(new_graph)
        self.period += 1


    # save to history
    def save_update_to_memory(self, updated_graph):
        # save new graph to history
        '''may need to make a history for t as well, if I end up not creating new nodes every iteration'''
        self.graph_history.append(updated_graph)


    # create new node, indexed as the next number in the list
    def new_node_birth(self, graph):
        current_nodes = graph.nodes()
        new_node = max(current_nodes) + 1 if len(current_nodes) > 0 else 1

        '''currently SES determined at birth, this can be changed'''
        new_node_SES = np.random.choice(['High', 'Low'], 1, p=[self.p_SES_high, 1-self.p_SES_high])[0]

        return (new_node, {'SES': new_node_SES})


    # find parent nodes connections
    def find_parent_nodes_connections(self, graph):
        # get the current existing nodes
        current_nodes = graph.nodes()

        # get m parent nodes
        parent_targets = np.random.choice(current_nodes, m, replace=False)

        # probability pm to connect to each m of them
        parents_edge_chance = np.random.choice([0,1], size=self.m, p=[1-self.pm, self.pm])

        # realised connections: cross product of the two vectors
        parent_connections = parent_targets * parents_edge_chance
        parent_connections = parent_connections[parent_connections > 0] # filter out the ones with no connection

        return parent_connections

    # find parent neighbour connections
    def find_parent_neighbour_connections(self, graph, parent_connections_list):
        # take the list of parents and find their (unique) neighbours
        parent_neighbours = np.unique(self.helper_functions().find_neighbours(graph, parent_connections_list))

        # to avoid issues: if the parents have in total less than n neighbours, take their number of neighbours instead
        n_possible_encounters = min(len(parent_neighbours), self.n)

        # get n parent neighbour nodes
        parent_neighbour_targets = np.random.choice(parent_neighbours, n_possible_encounters, replace=False)

        # probability pn to connect to each n of them
        parent_neighbours_edge_chance = np.random.choice([0,1], size = n_possible_encounters, p = (1-self.pn, self.pn))

        # realised connections: cross product of the two vectors
        parent_neighbours_connections = parent_neighbour_targets * parent_neighbours_edge_chance
        parent_neighbours_connections = parent_neighbours_connections[parent_neighbours_connections > 0] # filter out the ones with no connection


        return parent_neighbours_connections



    class biased_world:
        
        # new birth with SES
        def biased_node_birth(self, graph):
            current_nodes = graph.nodes()
            new_node = max(current_nodes) + 1 if len(current_nodes) > 0 else 1

            '''currently SES determined at birth, this can be changed'''
            new_node_SES = np.random.choice(['High', 'Low'], 1, p=[self.p_SES_high, 1-self.p_SES_high])[0]

            return (new_node, {'SES': new_node_SES})



        # find parent connections








        pass






    class helper_functions:
        '''useful functions'''
        # return the indicated attribute for a list of nodes
        def get_select_node_attributes(self, graph, attribute, node_list):
            attribute_dict_all_nodes = nx.get_node_attributes(graph, attribute)
            attribute_dict = {node: attribute_dict_all_nodes[node] for node in node_list}

            return attribute_dict

        # creates a list of edges based on targets 
        def edge_creator_helper(self, node, connection_targets):
            edges = [(node, connection_target) for connection_target in connection_targets]

            return edges

        # returns list of neighbours for all the nodes in the given list
        def find_neighbours(self, graph, node_list):
            if len(node_list) != 0:
                all_neighbours_list = np.concatenate(([list(graph[node]) for node in node_list]))
            else:
                all_neighbours_list = []

            return all_neighbours_list

        
        # check connection type of two connected nodes: cross(-class) or within(-class)
        def SES_edge_classifier(self, graph, node1, node2):    
            SES_values = list(self.get_select_node_attributes(graph, 'SES', [node1, node2]).values())
            link_type = 'within' if SES_values[0] == SES_values[1] else 'cross'

            return link_type

        # same, but now for all edges
        def SES_edge_classifier_all(self, graph):
            edges = graph.edges()

            list_edge_type = [self.SES_edge_classifier(graph, edge[0], edge[1]) for edge in edges]

            return list_edge_type




### NX drawing

In [ ]:


class sim_drawing(object):

    # can make a general one later
    def color_scale_from_attr(self, attr):


        return color_scale

    # assigns colour red if low SES, blue if high
    # NB: assumes that nodes are given in same order as when drawing
    def color_scale_SES(self, simulation, t=-1):
        SES_list = list(nx.get_node_attributes(simulation.graph_history[t], 'SES_High').values())
        color_scale_SES = ['blue' if SES == 'High' else 'red' for SES in SES_list]

        return color_scale_SES


    # kamada kawai network
    def draw_kamada_kawai(self, simulation, t=-1, color_scale = None):
        fig = plt.figure()

        title_suffix = simulation.period if t == -1 else t
        plt.title(f"Network Graph, t={title_suffix}")

        if color_scale == 'SES':
            cmap = self.color_scale_SES(simulation, t)
            nx.draw(simulation.graph_history[t], node_color = cmap, node_size = 10, cmap = cmap)

        else:
            nx.draw(simulation.graph_history[t], node_size = 10, cmap = cmap)


        plt.close()
        return fig










